In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 3.9MB 45.9MB/s 
     |████████████████████████████████| 450kB 40.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4e4f3776943a77056cc8b72e8202f692961ff6c0b53aa0487d5698f76ba9c8ab
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflo

In [0]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

2.1.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
questions=[]

for file in sorted(os.listdir("drive/My Drive/sparknotes/merged_except_romeo")):
    if file.endswith("_modern.snt.aligned"):
        filename_concat='drive/My Drive/sparknotes/merged_except_romeo/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        questions=questions+lines_modern


for file in sorted(os.listdir("drive/My Drive/enotes/merged")):
    if file.endswith("_modern.snt.aligned"):
        filename_concat='drive/My Drive/enotes/merged/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        questions=questions+lines_modern


In [0]:
answers=[]

for file in sorted(os.listdir("drive/My Drive/sparknotes/merged_except_romeo")):
    if file.endswith("_original.snt.aligned"):
        filename_concat='drive/My Drive/sparknotes/merged_except_romeo/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        answers=answers+lines_modern


for file in sorted(os.listdir("drive/My Drive/enotes/merged")):
    if file.endswith("_original.snt.aligned"):
        filename_concat='drive/My Drive/enotes/merged/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        answers=answers+lines_modern


In [0]:
len(questions)

29973

In [0]:
with open("drive/My Drive/dl/Output_new.txt", "w") as f:
  for i in range(len(questions)):
    f.write(answers[i]+'\t'+questions[i]+'\n')


In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w



# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs


# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word


def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)

   

    #npairs=[]
    #for i in pairs:
     # if (len(i[0].split())<=44):
      #  npairs.append(i)

    #pairs=npairs

    print(type(pairs))
    print((pairs[25000]))

    # index language using the class defined above    
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex(en for en, sp in pairs)
    
    # Vectorize the input and target languages
    
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    
    # English sentences
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]

    print(len(input_tensor))
    print(len(target_tensor))

    print(type(input_tensor))
    print(input_tensor[25000])
    print(target_tensor[25000])


    it=[]
    tt=[]

    for i in range(len(target_tensor)):
      if len(target_tensor[i])<40 and len(input_tensor[i])<40:
        it.append(input_tensor[i])
        tt.append(target_tensor[i])

    input_tensor=it
    target_tensor=tt
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [0]:
# Try experimenting with the size of that dataset
num_examples = 29972
path_to_file="drive/My Drive/dl/Output_new.txt"
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)


<class 'list'>
['<start> first , pyramus must draw a sword to kill himself which the ladies cannot abide . <end>', '<start> first , pyramus must draw a sword to kill himself , which the ladies cannot stand . <end>']
29972
29972
<class 'list'>
[5, 3907, 2, 8158, 6737, 3104, 7, 10343, 10681, 5646, 4909, 2, 11612, 10521, 5731, 1468, 9902, 3, 4]
[5, 4633, 2, 9420, 7845, 3641, 7, 11715, 12110, 6571, 5790, 13323, 11923, 6671, 1683, 22, 3, 4]


In [0]:
inp_lang, targ_lang, max_length_inp, max_length_targ

(<__main__.LanguageIndex at 0x7fba1f4b9940>,
 39,
 39)

In [0]:
print(input_tensor.shape)
print(max_length_inp)
print(target_tensor.shape)
print(max_length_targ)

(28842, 39)
39
(28842, 39)
39


In [0]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(25957, 25957, 2885, 2885)

In [0]:
type(input_tensor_train)

numpy.ndarray

In [0]:
target_tensor[78]

array([  5, 903, 681,   3,   4,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [0]:
input_tensor[78]

array([   5, 7405,  611,    3,    4,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
dataset

<BatchDataset shapes: ((64, 39), (64, 39)), types: (tf.int32, tf.int32)>

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)


In [0]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)


In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
EPOCHS = 2

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.2790
Epoch 1 Batch 100 Loss 0.3120
Epoch 1 Batch 200 Loss 0.3649
Epoch 1 Batch 300 Loss 0.4059
Epoch 1 Batch 400 Loss 0.2987
Epoch 1 Loss 0.3271
Time taken for 1 epoch 363.3114960193634 sec

Epoch 2 Batch 0 Loss 0.2212
Epoch 2 Batch 100 Loss 0.2799
Epoch 2 Batch 200 Loss 0.3813
Epoch 2 Batch 300 Loss 0.2562
Epoch 2 Batch 400 Loss 0.3314
Epoch 2 Loss 0.2810
Time taken for 1 epoch 366.4330463409424 sec



In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))


In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

**After 12 epochs....**

In [0]:
translate(u'have you killed Tybalt?', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> have you killed tybalt ? <end>
Predicted translation: hast thou slain tybalt ? <end> 


In [0]:
translate(u'look what you made me do !', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> look what you made me do ! <end>
Predicted translation: thou dost thou do me ! <end> 


In [0]:
translate(u'how you laugh when you lie ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> how you laugh when you lie <end>
Predicted translation: how dost thou been since thou lie ? <end> 


In [0]:
translate(u'What is wrong with you ? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what is wrong with you ? <end>
Predicted translation: what is amiss you ? <end> 


In [0]:
translate(u'Why, Romeo, are you crazy? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> why , romeo , are you crazy ? <end>
Predicted translation: why , romeo , thou wast mad ? <end> 


In [0]:
translate(u'bring her out to me . ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> bring her out to me . <end>
Predicted translation: bring her forth . <end> 


In [0]:
translate(u'i will hit you !', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> i will hit you ! <end>
Predicted translation: i will , beat thee , i will , i will , i will , i will , i will , i will , i will , i will , i will , i will , i will , 


In [0]:
translate(u'What fight was here? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what fight was here ? <end>
Predicted translation: what fray was here ? <end> 


In [0]:
translate(u'i ’ ll bite you by the ear for that joke . ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> i ll bite you by the ear for that joke . <end>
Predicted translation: i will tell thee by the joyful office for that jest . <end> 


In [0]:
translate(u'what do you want ?  ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what do you want ? <end>
Predicted translation: what would you ? <end> 


**After 10 epochs**

In [0]:
translate(u'have you killed Tybalt?', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> have you killed tybalt ? <end>
Predicted translation: hast thou slain tybalt ? <end> 


In [0]:
translate(u'look what you made me do .', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> look what you made me do . <end>
Predicted translation: thou seest thou do me . <end> 


In [0]:
translate(u'What is wrong with you ? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what is wrong with you ? <end>
Predicted translation: what is amiss with you ? <end> 


In [0]:
translate(u'Why, Romeo, are you crazy? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> why , romeo , are you crazy ? <end>
Predicted translation: why , hist ! <end> 


In [0]:
translate(u'bring her out to me . ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> bring her out to me . <end>
Predicted translation: bring her out , bring me . <end> 


In [0]:
translate(u'i will hit you !', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> i will hit you ! <end>
Predicted translation: i ll strike thee ! <end> 


In [0]:
translate(u'What fight was here? ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what fight was here ? <end>
Predicted translation: what fray was here ? <end> 


In [0]:
translate(u'i ’ ll bite you by the ear for that joke . ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> i ll bite you by the ear for that joke . <end>
Predicted translation: i ll fume with you a suitor for that jest . <end> 


In [0]:
translate(u'what do you want ?  ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> what do you want ? <end>
Predicted translation: what would you ? <end> 


**Reference:**

https://towardsdatascience.com/intuitive-understanding-of-attention-mechanism-in-deep-learning-6c9482aecf4f